# Reto 3

In [28]:
#Importar librerías
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from random import randrange
from sklearn import preprocessing

Se leen los datos utilizando readcsv de Pandas

In [29]:
#Leer el archivo con los datos
data_pulsar = pd.read_csv('./data/HTRU_2.csv', header=None)

Se separan los datos en "x" y "y". Se normaliza x

In [30]:
#La variable dependiente es la última columna, las independientes son las anteriores.
#Se normaliza las independientes y se convierte nuevamente en un DataFrame de Pandas
x= pd.DataFrame.from_records(preprocessing.normalize(data_pulsar.iloc[:,0:8]))
y= data_pulsar.iloc[:,8:9]

Se parten los datos en entrenamiento y test

In [31]:
#Se divide el archivo para entrenamiento y test. Se reserven 10000 datos para test
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 10000, random_state = 0)

In [32]:
#Se concatenan los datos de test
newData= pd.concat([xTrain,yTrain], axis = 1)

Se escogen aleatoriamente diferentes tamaños de datos para entrenar

In [33]:
#De estos datos concatenados se escogen aleatoriamente 100,500,1000 y 5000 para diferentes modelos. 
#De aquí se vuelven a separar en x y y
dataTrain1= newData.sample(100)
xTrain1= dataTrain1.iloc[:,0:8]
yTrain1= dataTrain1.iloc[:,8]
dataTrain2= newData.sample(500)
xTrain2= dataTrain2.iloc[:,0:8]
yTrain2= dataTrain2.iloc[:,8]
dataTrain3= newData.sample(1000)
xTrain3= dataTrain3.iloc[:,0:8]
yTrain3= dataTrain3.iloc[:,8]
dataTrain4= newData.sample(5000)
xTrain4= dataTrain4.iloc[:,0:8]
yTrain4= dataTrain4.iloc[:,8]

Se definen las funciones necesarias para realizar descenso de gradiente estocástico

In [34]:
#Definición de función sigmoide para usarse posteriormente
def sigmoid(x):
     return 1 / (1 + math.exp(-x))

In [35]:
#Función que calcula w mediante gradiente ascendente
#numErrors es el número de errores consecutivos cuya media debe ser menor que el threshold 
#para que se asuma que el algoritmo terminó.
#El threshold se define por la variable minError, este se escogió con ensayo y error
def gradient_asc(nX,nY,numErrors=12,minError=0.01):
    a=nX.to_numpy()
    y=nY.to_numpy()
    #Se concatena un 1 al final de las variables para representar la constante w0
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    #Se calcula la hessiana
    H=[[0] * wNumber]* wNumber
    for i in range(0, numData):
        xi= x[i]
        H+=xi.transpose()*xi
    #Se obtienen los valores propios y se calcula n como 2/lambdaMax
    #Este valor no corresponde exactamente a este modelo ya que el error cuadrático ahora tiene una
    #sigmoide, pero en la práctica descubrí que funciona
    ei=np.linalg.eigvals(H)
    n=2/max(ei)
    ws= [0.01] * wNumber
    error=0
    errors=[1]* numErrors
    j=0
    #Variable que representa la media mínima de los últimos numErrors datos
    minT=1
    #Mientras la media de los últimos numErrors errores sea mayor que el error mínimo
    while minT > minError:
        #Se escoge una fila de datos aleatoriamente
        i= randrange(numData)
        #Se calcula ws con el algoritmo LMS
        xs=(x[i]*ws).sum()
        g=sigmoid(xs)
        error=y[i]-g           
        ws= ws + n*error*x[i]
        #Se incluye en la lista de errores el último error. Como tal este no es el error de clasificación
        #pero es un buen estimativo para saber qué tan bien el modelo se acerca a predecir los datos
        errors[j]=abs(error)
        #En caso de llegar al final j vuelve a ser 0 para recorrer el arreglo en orden
        j=(j+1)%numErrors
        #Se guarda la media mínima en caso de ser menor que la última menor
        if minT>np.mean(errors):
            minT=np.mean(errors)
            #Se puede descomentar el siguiente print para observar como este valor disminuye
            #print(minT)
    return ws
    

Se calculan los modelos con las funciones anteriores

In [36]:
#Cálculo del modelo 1
logisticRegr1 = gradient_asc(xTrain1, yTrain1)
print(logisticRegr1)

[-3.3993097   0.29565536  0.3420489   1.76773919  2.21957763  2.54713457
 -0.10520867 -2.57410745 -0.91010248]


In [37]:
#Cálculo del modelo 2
logisticRegr2 = gradient_asc(xTrain2, yTrain2)
print(logisticRegr2)

[-7.85002327  6.33675937  1.80011025  6.96768749 -2.98734496  9.76361639
  1.0277272  -1.41837344 -0.78071249]


In [38]:
#Cálculo del modelo 3
logisticRegr3 = gradient_asc(xTrain3, yTrain3)
print(logisticRegr3)

[-5.90011621  5.66022996  1.767066    7.09477948 -1.49183464  7.3411227
  1.18757867 -1.32651542 -1.6903843 ]


In [39]:
#Cálculo del modelo 4
logisticRegr4 = gradient_asc(xTrain4, yTrain4)
print(logisticRegr4)

[-6.42799751  5.07921929  1.77756664  7.3912646  -2.43282826  8.90341286
  1.19259585 -1.28440255 -1.38913014]


Se construye una función de éxito para evaluar el modelo.

In [40]:
#Función que calcula el error final de clasificación. 
#Como tal lo que devuelve es el porcentaje de datos exitosamente clasificados
def calc_exito(nX,nY,w):
    a=nX.to_numpy()
    y=nY.to_numpy()
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    err=0
    for i in range(0, numData):
        prob1=sigmoid((x[i]*w).sum())
        #El porcentaje de error aumenta si se clasificó mal tanto como 1 cómo como 0
        if prob1>0.5 and y[i]==0:
            err=err+1
        #Se asigna el 0.5 como valor inclusivo hacia el 0 dado que el porcentaje de 0's es mucho mayor.
        if prob1<=0.5 and y[i]==1:
            err=err+1
    return 1-(err/numData)
    

Se evalúa el modelo con la función personalizada de éxito

In [41]:
#Se calcula el porcentaje de éxito de clasificación para el modelo 1
exito1 = calc_exito(xTest, yTest,logisticRegr1)
print(exito1)

0.9532


In [42]:
#Se calcula el porcentaje de éxito de clasificación para el modelo 2
exito2 = calc_exito(xTest, yTest,logisticRegr2)
print(exito2)

0.971


In [43]:
#Se calcula el porcentaje de éxito de clasificación para el modelo 3
exito3 = calc_exito(xTest, yTest,logisticRegr3)
print(exito3)

0.9694


In [44]:
#Se calcula el porcentaje de éxito de clasificación para el modelo 4
exito4 = calc_exito(xTest, yTest,logisticRegr4)
print(exito4)

0.9701


# Resultados
Se puede observar que el porcentaje de éxito de clasificación es bastante alto. El porcentaje de éxito es casi tan alto como el de la librería sklearn. Su valor no llegó a ser el mismo posiblemente por utilizar el algoritmo de descenso de gradiente estocástico en vez de calcular el mínimo exacto. Se puede observar que el modeo con mayor porcentaje de éxito de clasificación es el primero. Esto es contraintuitivo ya que el primer modelo es el que tiene menor número de datos. Sin embargo puede que dentro de la aleatoriedad, el modelo pudo quedar con un porcentaje mayor de y=1, lo cual ayuda a que el modelo no quede sesgado hacia los 0's y no se aprenda los datos. Sin embargo, en general todos los modelos tienen un porcentaje de clasificación similar